In [5]:
from file_cache import *

from torchvision import models
from fastai.vision import *
from fastai.callbacks.hooks import *
def unet_dynamic(arch: Callable = models.resnet34, pretrained: bool = True, blur_final: bool = True,
                 norm_type: Optional[NormType] = NormType, split_on: Optional[SplitFuncOrIdxList] = None,
                 blur: bool = False,
                 self_attention: bool = False, y_range: Optional[Tuple[float, float]] = None, last_cross: bool = True,
                 bottle: bool = False, cut: Union[int, Callable] = None,
                 n_classes=2, img_size=(224, 224), in_channels=1,
                 **learn_kwargs: Any) -> Learner:
    "Build Unet learner from `data` and `arch`."
    "blur: do maxpolling or not"
    #import ipdb
    #ipdb.set_trace()
    body = create_body(arch, pretrained, cut)
    model = to_device(
        models.unet.DynamicUnet(body, n_classes=n_classes, img_size=img_size, blur=blur, blur_final=blur_final,
                                self_attention=self_attention, y_range=y_range, norm_type=norm_type,
                                last_cross=last_cross,
                                bottle=bottle), 'cuda')
    return model


#unet_dynamic()

2020-06-07 00:13:42,974 util_log.py[153] INFO Start the program at:amax7, 127.0.1.1, with:Load module


File_cache: Adjust notebook work fold to:/share/felix/pj/brain_seg/


In [ ]:
def has_pool_type(m):
    if is_pool_type(m): return True
    for l in m.children():
        if has_pool_type(l): return True
    return False

ll = list(enumerate(models.resnet50().children()))
next(i for i,o in reversed(ll) if has_pool_type(o))

In [2]:
model = models.resnet50()
test=torch.rand(2,3,224,224)

model = nn.Sequential(*list(model.children())[:-3])
dummy_batch(model, size=(512, 512)).shape

#flatten_model(model)

torch.Size([1, 3, 512, 512])

(64, 64)


tensor([[[[10.2573,  9.4800,  5.5673,  9.0349],
          [ 8.8604,  8.3111,  5.1963,  5.8235],
          [ 9.9536,  9.0420, 10.5154,  6.9228],
          [ 6.4321,  2.2345,  0.9525,  0.7423]],

         [[ 4.9615,  1.4991,  5.2210,  5.5049],
          [14.4554,  4.3003,  7.5125,  3.6751],
          [14.2390,  5.7331,  3.7914,  2.8598],
          [11.9957,  6.2396,  2.8204,  0.8718]],

         [[ 3.9049,  1.7494,  1.1136,  1.2751],
          [ 3.4082,  3.6669,  3.4593,  3.7076],
          [ 2.2069,  8.9022,  7.5163,  3.9783],
          [ 6.6491, 11.0994, 10.5415,  5.4013]],

         ...,

         [[ 5.0409,  7.7732,  9.1715,  4.5158],
          [ 8.8941, 16.9469, 13.8383,  9.1247],
          [10.4952, 20.2004, 18.1006,  9.1283],
          [ 3.8249,  8.4302,  7.8514,  3.5858]],

         [[10.4764, 10.3584,  7.6579,  8.5185],
          [ 7.2708, 11.6850, 13.3774, 12.5276],
          [ 6.4262, 10.7750, 13.1700, 11.2665],
          [ 4.1295,  7.5469, 10.7780, 12.0054]],

         [[ 0.0

In [3]:
from efficientnet_pytorch import EfficientNet
def eff(*args, **kwargs):
    model = EfficientNet.from_pretrained('efficientnet-b0')
    return model
import ipdb
#ipdb.set_trace()
#unet_dynamic(eff)
eff()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [4]:



# def dummy_batch(m: nn.Module, size:tuple=(64,64), ch_in=3)->Tensor:
#     "Create a dummy batch to go through `m` with `size`."
#     print('======ch_in', ch_in)
#     #ch_in = in_channels(m)
    
#     return one_param(m).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)


def dummy_eval(m:nn.Module, size:tuple=(64,64)):
    "Pass a `dummy_batch` in evaluation mode in `m` with `size`."
    print(size)

    return m.eval()(dummy_batch(m, size))

def model_sizes(m:nn.Module, size:tuple=(64,64))->Tuple[Sizes,Tensor,Hooks]:
    "Pass a dummy input through the model `m` to get the various sizes of activations."
    with hook_outputs(m) as hooks:
        import ipdb
        #ipdb.set_trace()
        x = m.eval()(torch.rand(1,3,224,224))
        print('=======', x.shape)
        return [o.stored.shape for o in hooks]
    

In [ ]:
b /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/torch_core.py:262

In [5]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')
#model = models.resnet50()
#test=torch.rand(2,3,224,224)
model = nn.Sequential(*list(model.children()))
model_sizes(model, size=(512, 512))#.shape

#dummy_eval(model, size=(512, 512)).shape

Loaded pretrained weights for efficientnet-b0


NotImplementedError: 

In [10]:
model = EfficientNet.from_pretrained('efficientnet-b0')
#model = models.resnet50()
#test=torch.rand(2,3,224,224)
# model = nn.Sequential(*list(model.children()))
# model(torch.rand(1,3,224,224))

NameError: name 'EfficientNet' is not defined

In [6]:
import sys
sys.path.insert(0, '/home/felix/pj/EfficientUnet-PyTorch')
 

In [5]:
from efficientunet.efficientnet  import EfficientNet
import torch
from collections import OrderedDict
from torch import nn
from easydict import EasyDict as edict
import pandas as pd
 
#model = EfficientNet.encoder('efficientnet-b5')
#from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b0') 
import torch

get_select_layer(model)

([13, 37, 61, 133, 193], [96, 144, 240, 672, 1280])

In [109]:
img_size = [ 7*(2**i) for i in range(6)]
xx = tmp.loc[(tmp.layer.isin( ['BatchNorm2d', 'Swish'])) & (tmp.w.isin(img_size))]#.layer.value_counts()
pass

xx

,sn,layer,c,w,h
1,1,BatchNorm2d,48,112,112
2,2,Swish,48,112,112
4,4,BatchNorm2d,48,112,112
5,5,Swish,48,112,112
10,10,BatchNorm2d,24,112,112
...,...,...,...,...,...
454,454,BatchNorm2d,3072,7,7
455,455,Swish,3072,7,7
460,460,BatchNorm2d,512,7,7
463,463,BatchNorm2d,2048,7,7


([144, 240, 384, 1056, 2048], [31, 91, 151, 319, 463])

In [108]:
print(tmp.loc[(tmp.layer) & (tmp.w.isin(img_size))].drop_duplicates(['w'], keep='last').c.to_list())


TypeError: unsupported operand type(s) for &: 'str' and 'bool'

In [87]:
list(reversed([96, 144, 240, 672, 1280]))

[1280, 672, 240, 144, 96]

In [40]:

def get_blocks_to_be_concat(model, x):
    shapes = set()
    blocks = OrderedDict()
    hooks = []
    count = 0

    def register_hook(module):

        def hook(module, input, output):
            try:
                nonlocal count
                if module.name == f'blocks_{count}_output_batch_norm':
                    count += 1
                    shape = output.size()[-2:]
                    if shape not in shapes:
                        shapes.add(shape)
                        blocks[module.name] = output
                        print(module.name, output.shape)

                elif module.name == 'head_swish':
                    # when module.name == 'head_swish', it means the program has already got all necessary blocks for
                    # concatenation. In my dynamic unet implementation, I first upscale the output of the backbone,
                    # (in this case it's the output of 'head_swish') concatenate it with a block which has the same
                    # Height & Width (image size). Therefore, after upscaling, the output of 'head_swish' has bigger
                    # image size. The last block has the same image size as 'head_swish' before upscaling. So we don't
                    # really need the last block for concatenation. That's why I wrote `blocks.popitem()`.
                    blocks.popitem()
                    blocks[module.name] = output
                    print(module.name, output.shape)

            except AttributeError:
                pass

        if (
                not isinstance(module, nn.Sequential)
                and not isinstance(module, nn.ModuleList)
                and not (module == model)
        ):
            hooks.append(module.register_forward_hook(hook))

    # register hook
    model.apply(register_hook)

    # make a forward pass to trigger the hooks
    model(x)

    # remove these hooks
    for h in hooks:
        h.remove()
    
    print(type(blocks))
    for item in blocks.items():
        print(item[0], item[1].shape)
    #return blocks

model = EfficientNet.encoder('efficientnet-b5')
get_blocks_to_be_concat(model, torch.rand(1,3,224,224))
pass

blocks_0_output_batch_norm torch.Size([1, 24, 112, 112])
blocks_3_output_batch_norm torch.Size([1, 40, 56, 56])
blocks_8_output_batch_norm torch.Size([1, 64, 28, 28])
blocks_13_output_batch_norm torch.Size([1, 128, 14, 14])
blocks_27_output_batch_norm torch.Size([1, 304, 7, 7])
head_swish torch.Size([1, 2048, 7, 7])
<class 'collections.OrderedDict'>
blocks_0_output_batch_norm torch.Size([1, 24, 112, 112])
blocks_3_output_batch_norm torch.Size([1, 40, 56, 56])
blocks_8_output_batch_norm torch.Size([1, 64, 28, 28])
blocks_13_output_batch_norm torch.Size([1, 128, 14, 14])
head_swish torch.Size([1, 2048, 7, 7])


In [38]:
model(test).sum()

tensor(-1.6385, grad_fn=<SumBackward0>)

In [37]:
model(test).sum()

tensor(-1.6385, grad_fn=<SumBackward0>)

In [ ]:
b /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/vision/models/unet.py:43

In [5]:
from efficientnet_pytorch import EfficientNet
def eff(*args, **kwargs):
    model = EfficientNet.from_pretrained('efficientnet-b0')
    return model
import ipdb
#ipdb.set_trace()
unet_dynamic(eff)

--Return--
None
> <ipython-input-5-1fe1e2bd3f4e>(6)<module>()
      5 import ipdb
----> 6 ipdb.set_trace()
      7 unet_dynamic(eff)

ipdb> b /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/vision/models/unet.py:43
Breakpoint 1 at /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/vision/models/unet.py:43
ipdb> c
Loaded pretrained weights for efficientnet-b0
> /home/aladdin1/anaconda3/lib/python3.7/site-packages/fastai/vision/models/unet.py(43)__init__()
     42         imsize = img_size
1--> 43         sfs_szs = model_sizes(encoder, size=imsize)
     44         sfs_idxs = list(reversed(_get_sfs_idxs(sfs_szs)))

ipdb> type(encoder)
<class 'torch.nn.modules.container.Sequential'>
ipdb> encoder
Sequential(
  (0): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (

ipdb> c


IndexError: tuple index out of range

In [4]:
create_body(eff, pretrained=True, cut=None)

Loaded pretrained weights for efficientnet-b0


Sequential(
  (0): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (2): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1

# Branding New

In [25]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from easydict import EasyDict as edict
import pandas as pd


def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )


def up_conv(in_channels, out_channels):
    return nn.ConvTranspose2d(
        in_channels, out_channels, kernel_size=2, stride=2
    )


class EfficientUnet(nn.Module):
    def __init__(self, encoder, out_channels=2, concat_input=True):
        super().__init__()

        self.layer_index, self.channel_list = self.get_unet_config(encoder)

        self.encoder = encoder
        self.concat_input = concat_input

        self.up_conv1 = up_conv(self.channel_list[-1], 512)
        self.double_conv1 = double_conv(self.size[0], 512)
        self.up_conv2 = up_conv(512, 256)
        self.double_conv2 = double_conv(self.size[1], 256)
        self.up_conv3 = up_conv(256, 128)
        self.double_conv3 = double_conv(self.size[2], 128)
        self.up_conv4 = up_conv(128, 64)
        self.double_conv4 = double_conv(self.size[3], 64)

        if self.concat_input:
            self.up_conv_input = up_conv(64, 32)
            self.double_conv_input = double_conv(self.size[4], 32)

        self.final_conv = nn.Conv2d(self.size[5], out_channels, kernel_size=1)


    def get_blocks_to_be_concat(self, model, x):
        shapes = set()
        blocks = OrderedDict()
        hooks = []
        count = 0

        def register_hook(module):

            def hook(module, input, output):
                try:
                    nonlocal count

                    if count in self.layer_index:
                        blocks[count] = output
                    count += 1
                except AttributeError:
                    pass

            hooks.append(module.register_forward_hook(hook))

        # register hook
        model.apply(register_hook)

        # make a forward pass to trigger the hooks
        model(x)

        # remove these hooks
        for h in hooks:
            h.remove()

        return blocks

    def get_unet_config(self, model, x=torch.rand(1, 3, 512, 512)):
        shapes = []
        blocks = OrderedDict()
        hooks = []
        count = 0
        channel = edict()
        res = []

        def register_hook(module):

            def hook(module, input, output):
                try:
                    nonlocal count

                    # print(output.shape)
                    if len(output.shape) == 4:
                        b, c, w, h = output.shape
                        res.append((count, type(module).__name__, c, w, h))
                    else:
                        pass
                        # print( count, output.shape,)

                    # channel[module.name]= output.size()[1]
                    count += 1
                except AttributeError:
                    print('Error:', type(module))
                    pass

            hooks.append(module.register_forward_hook(hook))

        # register hook
        model.apply(register_hook)

        # make a forward pass to trigger the hooks
        model(x)

        # remove these hooks
        for h in hooks:
            h.remove()

        tmp = pd.DataFrame(res, columns=['sn', 'layer', 'c', 'w', 'h'])
        #print(tmp)
        img_size = [x.shape[-1] // (2 ** i) for i in range(6)]
        tmp = tmp.loc[(tmp.layer == 'BatchNorm2d') & (tmp.w.isin(img_size))].drop_duplicates(['w'], keep='last')
        print(tmp)
        print('layer_list, channel', list(tmp.sn), list(tmp.c))
        return list(tmp.sn), list(tmp.c)

    @property
    def n_channels(self):
        return self.channel_list[-1]

    @property
    def size(self):

        return [512 + self.channel_list[-2],
                256 + self.channel_list[-3],
                128 + self.channel_list[-4],
                64 + self.channel_list[-5],
                35,
                32]

    def forward(self, x):
        input_ = x

        blocks = self.get_blocks_to_be_concat(self.encoder, x)
        _, x = blocks.popitem()

        x = self.up_conv1(x)
        x = torch.cat([x, blocks.popitem()[1]], dim=1)
        x = self.double_conv1(x)

        x = self.up_conv2(x)
        x = torch.cat([x, blocks.popitem()[1]], dim=1)
        x = self.double_conv2(x)

        x = self.up_conv3(x)
        x = torch.cat([x, blocks.popitem()[1]], dim=1)
        x = self.double_conv3(x)

        x = self.up_conv4(x)
        x = torch.cat([x, blocks.popitem()[1]], dim=1)
        x = self.double_conv4(x)

        if self.concat_input:
            x = self.up_conv_input(x)
            x = torch.cat([x, input_], dim=1)
            x = self.double_conv_input(x)

        x = self.final_conv(x)

        return x


def get_efficientunet(out_channels=5, concat_input=True, pretrained=True):
    from efficientnet_pytorch import EfficientNet
    encoder = EfficientNet.from_pretrained('efficientnet-b0')
    # encoder = EfficientNet.encoder('efficientnet-b0', pretrained=pretrained)
    model = EfficientUnet(encoder, out_channels=out_channels, concat_input=concat_input)
    return model


 
tmp = get_efficientunet()
tmp(torch.rand(1, 3, 224, 224)).shape

Loaded pretrained weights for efficientnet-b0
      sn        layer     c    w    h
19    19  BatchNorm2d    96  256  256
53    53  BatchNorm2d   144  128  128
87    87  BatchNorm2d   240   64   64
189  189  BatchNorm2d   672   32   32
274  274  BatchNorm2d  1280   16   16
layer_list, channel [19, 53, 87, 189, 274] [96, 144, 240, 672, 1280]


torch.Size([1, 5, 224, 224])

In [12]:

pass

Loaded pretrained weights for efficientnet-b0
layer_list, channel [19, 53, 87, 189, 274] [96, 144, 240, 672, 1280]
